In [ ]:
Skip to content
MDAnalysis
/
WorkshopHackathon2018
Code
Issues
4
Pull requests
Actions
Projects
1
Wiki
More
WorkshopHackathon2018/03_Tutorial1/MDAnalysis_Workshop_Part_1_solutions.ipynb
@richardjgowers
richardjgowers restored part 1 notebooks
 History
 1 contributor
2164 lines (2164 sloc)  70.6 KB
  
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### This notebook introduces the fundamental objects of `MDAnalysis`:\n",
    "\n",
    " - the `Universe`,\n",
    " - `AtomGroup`, and `Atom` objects.\n",
    " - selecting and manipulating `AtomGroup`s\n",
    " - `Residue`s and `Segment`s\n",
    " - `Bond` `Angle` and `Dihedral`"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 1. Fundamental objects\n",
    "\n",
    "## `Universes` and `AtomGroups`\n",
    "\n",
    "> \"If you wish to make an apple pie from scratch, you must first invent the Universe.\" \n",
    "\n",
    "> ~ Carl Sagan"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "First, we need to import `MDAnalysis`, giving us access to all the components in its namespace:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0+unknown\n",
      "0.5.0+4.g1c9dda0\n"
     ]
    }
   ],
   "source": [
    "import MDAnalysis as mda\n",
    "import MDAnalysisData as data\n",
    "print(mda.__version__)\n",
    "print(data.__version__)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "One of the most fundamental objects in the `MDAnalysis` data model is the `Universe` object.\n",
    "A `Universe` can be thought of as an interface to all the data of a simulation;\n",
    "it contains all of a simulations' topology information (names, charges, masses etc) at the least,\n",
    "but usually also includes trajectory information (positions, velocities etc) as well."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "In order to do anything, we do need some actual molecular dynamics data to work with.\n",
    "You should have downloaded a dataset package for this workshop;\n",
    "Let's load a dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      ".. -*- coding: utf-8 -*-\n",
      "\n",
      ".. _`adk-equilibrium-dataset`:\n",
      "\n",
      "AdK equilibrium trajectory dataset\n",
      "==================================\n",
      "\n",
      "MD trajectory of apo adenylate kinase with CHARMM27 force field and\n",
      "simulated with explicit water and ions in NPT at 300 K and 1\n",
      "bar. Saved every 240 ps for a total of 1.004 µs. Produced on PSC\n",
      "Anton. The trajectory only contains the protein while all solvent was\n",
      "stripped in order to reduce the trajectory size. The whole protein was\n",
      "superimposed on the CORE domain of AdK by RMSD fitting.\n",
      "\n",
      "The topology is contained in the PSF file (CHARMM format). The\n",
      "trajectory is contained in the DCD file (CHARMM/NAMD format).\n",
      "\n",
      "\n",
      "Notes\n",
      "-----\n",
      "\n",
      "Data set characteristics:\n",
      "\n",
      " :size: 161 MB\n",
      " :number of trajectories: 1\n",
      " :number of frames:  4187\n",
      " :number of particles: 3341\n",
      " :creator: Sean Seyler\n",
      " :URL:  `10.6084/m9.figshare.5108170.v1 <https://doi.org/10.6084/m9.figshare.5108170.v1>`_\n",
      " :license: `CC-BY 4.0 <https://creativecommons.org/licenses/by/4.0/legalcode>`_\n",
      " :reference: [Seyler2017]_\n",
      "    \n",
      "\n",
      ".. [Seyler2017]  Seyler, Sean; Beckstein, Oliver (2017): Molecular dynamics\n",
      "\t   trajectory for benchmarking\n",
      "\t   MDAnalysis. figshare. Fileset. doi:\n",
      "\t   `10.6084/m9.figshare.5108170.v1\n",
      "\t   <https://doi.org/10.6084/m9.figshare.5108170.v1>`_\n",
      "\n",
      "\n"
     ]
    }
   ],
   "source": [
    "adk_dataset = data.datasets.fetch_adk_equilibrium()\n",
    "print(adk_dataset['DESCR'])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "To make a Universe, we need at the very least a topology file. We'll use one for a simulation system that includes adenylate kinase with the water removed:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "<Universe with 3341 atoms>\n"
     ]
    }
   ],
   "source": [
    "u = mda.Universe(adk_dataset['topology'], adk_dataset['trajectory'])\n",
    "\n",
    "print(u)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We now have a `Universe` object. Since the topology (PSF) file we loaded contained both atom identities and bond information, the `Universe` is able to access these details.\n",
    "\n",
    "We can access all atoms in the `Universe` through the `Universe.atoms` attribute.\n",
    "This returns an `AtomGroup` which is probably the most important class we will learn about."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "MDAnalysis.core.groups.AtomGroup"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag = u.atoms\n",
    "type(ag)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We previously learnt about `numpy` arrays, which represent a series of numbers.\n",
    "\n",
    "An `AtomGroup` is like an array of atoms, and offers access to the data of these atoms through various attributes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([   0,    1,    2, ..., 3338, 3339, 3340])"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.indices"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['N', 'HT1', 'HT2', ..., 'CA', 'HA1', 'HA2'], dtype=object)"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.names"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['MET', 'MET', 'MET', ..., 'GLY', 'GLY', 'GLY'], dtype=object)"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.resnames"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([  1,   1,   1, ..., 214, 214, 214])"
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.resids"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([-0.30000001,  0.33000001,  0.33000001, ..., -0.02      ,\n",
       "        0.09      ,  0.09      ])"
      ]
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.charges"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([14.007,  1.008,  1.008, ..., 12.011,  1.008,  1.008])"
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.masses"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['NH3', 'HC', 'HC', ..., 'CT2', 'HB', 'HB'], dtype=object)"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.types"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "All of these attributes of an `AtomGroup` return numpy arrays of the same length as the `AtomGroup` itself;\n",
    "that is, each element corresponds to each atom in the `AtomGroup`, in order.\n",
    "\n",
    "In general, `MDAnalysis` will try and extract as much information as possible from the files given to `Universe`."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Working with individual atoms\n",
    "\n",
    "By slicing an `AtomGroup` we can access individual `Atom` objects.\n",
    "These `Atom` objects will have singular versions of the various attributes of `AtomGroup`s.\n",
    "\n",
    "In general working with individual `Atom` objects is discouraged as it is inefficient and will lead to poor performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "<Atom 1: N of type NH3 of resname MET, resid 1 and segid ADK>\n"
     ]
    }
   ],
   "source": [
    "a = u.atoms[0]\n",
    "print(a)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "name: N\n",
      "resid: 1\n",
      "resname: MET\n"
     ]
    }
   ],
   "source": [
    "print(\"name:\", a.name)\n",
    "print(\"resid:\", a.resid)\n",
    "print(\"resname:\", a.resname)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 2. Selecting atoms\n",
    "\n",
    "It is also rare that we want to operate on all atoms in the system!\n",
    "\n",
    "`MDAnalysis` offers a few different ways to select atoms,\n",
    "in this section we will go over the most useful methods."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2.1 Numpy style selections\n",
    "\n",
    "As previously mentioned, an `AtomGroup` is like an array of atoms,\n",
    "and therefore we can slice it exactly like we would slice a `numpy` array\n",
    "\n",
    "### Fancy indexing\n",
    "\n",
    "The simplest option to select specific atom is to use fancy indexing. You can specify the atoms in a list"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 4 atoms>"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms[[1, 4, 5 , 0]]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "or as a range"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 9 atoms>"
      ]
     },
     "execution_count": 16,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms[1:10]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Boolean indexing\n",
    "\n",
    "You can also create an array with `True`/`False` values of the same length as the `AtomGroup`. Every atom for which the array is set to `True` will be selected.\n",
    "\n",
    "We can create such a boolean array by doing a comparison of a `numpy` array:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "selection array =  [False False False ...  True  True  True]\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 141 atoms>"
      ]
     },
     "execution_count": 17,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "selection_ar = u.atoms.resnames == 'GLY'\n",
    "print(\"selection array = \", selection_ar)\n",
    "u.atoms[selection_ar]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "or shorter:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 141 atoms>"
      ]
     },
     "execution_count": 18,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms[u.atoms.resnames == 'GLY']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2.2 Selection Strings and `select_atoms`\n",
    "\n",
    "We've already seen that complex selections can be performed on `AtomGroups` using numpy style indexing.\n",
    "However, `MDAnalysis` also features a CHARMM-style atom selection mechanism that is often more convenient.\n",
    "We can consult the docstring for `select_atoms` to see the available selection keywords"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [],
   "source": [
    "u.atoms.select_atoms?"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Although boolean selections work well enough for selecting out atoms from AtomGroups, the selection language makes more complex selections possible with probably less effort.\n",
    "\n",
    "For example, we can create selections of all atoms in a particular residue type:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 474 atoms>"
      ]
     },
     "execution_count": 20,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "acidic = u.select_atoms(\"resname GLU or resname ASP\")\n",
    "acidic"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "For name like selections, we can also use the `*` symbol to indicate a wildcard selection.\n",
    "\n",
    "Here for example `name OD*` would select `OD1, OD2, OD3` etc"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 70 atoms>"
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "acidic_o = acidic.select_atoms('name OD* or name OE*')\n",
    "acidic_o"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "As a shortcut, multiple values can be given and these will be implicitly OR'd together.\n",
    "For example to select all atoms with name NZ OR NH* in residues named LYS or ARG:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 44 atoms>"
      ]
     },
     "execution_count": 22,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "basic_n = u.select_atoms(\"(resname LYS ARG) and (name NZ NH*)\")\n",
    "basic_n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Geometric selections\n",
    "\n",
    "The `select_atoms` method also has various geometric keywords that make selecting atoms based on geometric criteria much easier.\n",
    "\n",
    "If we want only residues that are involved in salt bridges, we can use our `AtomGroups` as part of additional selections. We can also use the `around` selection operator to specify only atoms within 4 angstroms. At the end we can get the full residues back."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [],
   "source": [
    "acidic = u.select_atoms(\"group acidic and around 4 group basic\", acidic=acidic_o, basic=basic_n)\n",
    "basic = u.select_atoms(\"group basic and around 4 group acidic\", acidic=acidic_o, basic=basic_n)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 20 atoms>"
      ]
     },
     "execution_count": 24,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "acidic"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2.3 Set operations\n",
    "\n",
    "`AtomGroup`s can also be combined using `&` for \"and\" and `|` for \"or\":"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "1656 312\n",
      "143\n"
     ]
    }
   ],
   "source": [
    "heavy = u.atoms[u.atoms.masses > 10]\n",
    "arg = u.atoms[u.atoms.resnames == 'ARG']\n",
    "print(len(heavy), len(arg))\n",
    "print(len(heavy & arg))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "These two selections are identical:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "True"
      ]
     },
     "execution_count": 26,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag1 = u.atoms.select_atoms('resname LYS ARG')\n",
    "\n",
    "ag2 = u.atoms.select_atoms('resname LYS') | u.atoms.select_atoms('resname ARG')\n",
    "\n",
    "ag1 == ag2"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We can also "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "True"
      ]
     },
     "execution_count": 27,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "no_H = u.atoms.select_atoms('resname LYS ARG and not name H*')\n",
    "\n",
    "no_H == u.atoms.select_atoms('resname LYS ARG') - u.atoms.select_atoms('name H*')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "By design, an `AtomGroup` can have repeats of the same atom, for example through this selection:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 7 atoms>"
      ]
     },
     "execution_count": 28,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag = u.atoms[[0, 0, 1, 2, 4, 4, 5]]\n",
    "ag"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The `unique` property will return a version of the `AtomGroup` with only one of each Atom:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<AtomGroup with 5 atoms>"
      ]
     },
     "execution_count": 29,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "ag.unique"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Challenge:\n",
    "\n",
    "- #### count the number of glycine residues in the protein\n",
    "- #### select all Nitrogen atoms within 5.0A of a carbon atom\n",
    "- #### select all oxygens that are bonded to a alpha Carbon"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 3. `ResidueGroups` and `SegmentGroups`"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The `Universe` also gives higher-order topology objects, including `ResidueGroups` and `SegmentGroups`. We can access all residues in the `Universe` with:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<ResidueGroup with 214 residues>"
      ]
     },
     "execution_count": 33,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.residues"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "And all segments with:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 34,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<SegmentGroup with 1 segment>"
      ]
     },
     "execution_count": 34,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.segments"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "`ResidueGroups` and `SegmentGroups` also behave similarly to `AtomGroups`, with many of their methods returning `numpy` arrays with each element corresponding to a single residue or segment, respectively."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 35,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['MET', 'ARG', 'ILE', 'ILE', 'LEU', 'LEU', 'GLY', 'ALA', 'PRO',\n",
       "       'GLY', 'ALA', 'GLY', 'LYS', 'GLY', 'THR', 'GLN', 'ALA', 'GLN',\n",
       "       'PHE', 'ILE', 'MET', 'GLU', 'LYS', 'TYR', 'GLY', 'ILE', 'PRO',\n",
       "       'GLN', 'ILE', 'SER', 'THR', 'GLY', 'ASP', 'MET', 'LEU', 'ARG',\n",
       "       'ALA', 'ALA', 'VAL', 'LYS', 'SER', 'GLY', 'SER', 'GLU', 'LEU',\n",
       "       'GLY', 'LYS', 'GLN', 'ALA', 'LYS', 'ASP', 'ILE', 'MET', 'ASP',\n",
       "       'ALA', 'GLY', 'LYS', 'LEU', 'VAL', 'THR', 'ASP', 'GLU', 'LEU',\n",
       "       'VAL', 'ILE', 'ALA', 'LEU', 'VAL', 'LYS', 'GLU', 'ARG', 'ILE',\n",
       "       'ALA', 'GLN', 'GLU', 'ASP', 'CYS', 'ARG', 'ASN', 'GLY', 'PHE',\n",
       "       'LEU', 'LEU', 'ASP', 'GLY', 'PHE', 'PRO', 'ARG', 'THR', 'ILE',\n",
       "       'PRO', 'GLN', 'ALA', 'ASP', 'ALA', 'MET', 'LYS', 'GLU', 'ALA',\n",
       "       'GLY', 'ILE', 'ASN', 'VAL', 'ASP', 'TYR', 'VAL', 'LEU', 'GLU',\n",
       "       'PHE', 'ASP', 'VAL', 'PRO', 'ASP', 'GLU', 'LEU', 'ILE', 'VAL',\n",
       "       'ASP', 'ARG', 'ILE', 'VAL', 'GLY', 'ARG', 'ARG', 'VAL', 'HSE',\n",
       "       'ALA', 'PRO', 'SER', 'GLY', 'ARG', 'VAL', 'TYR', 'HSE', 'VAL',\n",
       "       'LYS', 'PHE', 'ASN', 'PRO', 'PRO', 'LYS', 'VAL', 'GLU', 'GLY',\n",
       "       'LYS', 'ASP', 'ASP', 'VAL', 'THR', 'GLY', 'GLU', 'GLU', 'LEU',\n",
       "       'THR', 'THR', 'ARG', 'LYS', 'ASP', 'ASP', 'GLN', 'GLU', 'GLU',\n",
       "       'THR', 'VAL', 'ARG', 'LYS', 'ARG', 'LEU', 'VAL', 'GLU', 'TYR',\n",
       "       'HSE', 'GLN', 'MET', 'THR', 'ALA', 'PRO', 'LEU', 'ILE', 'GLY',\n",
       "       'TYR', 'TYR', 'SER', 'LYS', 'GLU', 'ALA', 'GLU', 'ALA', 'GLY',\n",
       "       'ASN', 'THR', 'LYS', 'TYR', 'ALA', 'LYS', 'VAL', 'ASP', 'GLY',\n",
       "       'THR', 'LYS', 'PRO', 'VAL', 'ALA', 'GLU', 'VAL', 'ARG', 'ALA',\n",
       "       'ASP', 'LEU', 'GLU', 'LYS', 'ILE', 'LEU', 'GLY'], dtype=object)"
      ]
     },
     "execution_count": 35,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.residues.resnames"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 36,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['ADK'], dtype=object)"
      ]
     },
     "execution_count": 36,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.segments.segids"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We can more easily get the number of glycine residues using a `ResidueGroup`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 37,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "20"
      ]
     },
     "execution_count": 37,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.residues[u.residues.resnames == 'GLY'].n_residues"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "or perhaps more Pythonic:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 38,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "20"
      ]
     },
     "execution_count": 38,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(u.residues[u.residues.resnames == 'GLY'])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 4. Accessing coordinates\n",
    "\n",
    "The most important attribute of your atoms is undoubtedly their positions!\n",
    "\n",
    "Again, the position information is made available via an `AtomGroup` in the `.positions` attribute:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 52,
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[ -8.376492  ,  12.685615  ,   2.507646  ],\n",
       "       [ -8.808646  ,  14.00077   ,   2.738939  ],\n",
       "       [ -9.307219  ,  14.329495  ,   4.0661473 ],\n",
       "       [ -9.580463  ,  13.360274  ,   4.983222  ],\n",
       "       [ -8.634222  ,  11.650653  ,   3.503505  ],\n",
       "       [ -9.068992  ,  12.0633745 ,   4.7793655 ],\n",
       "       [ -2.3986118 ,  13.009312  ,   7.920652  ],\n",
       "       [ -2.6920655 ,  13.299093  ,   9.301532  ],\n",
       "       [ -3.8000007 ,  12.6485405 ,   9.878869  ],\n",
       "       [ -4.637756  ,  11.742239  ,   9.104898  ],\n",
       "       [ -3.225632  ,  12.14919   ,   7.249131  ],\n",
       "       [ -4.361382  ,  11.587018  ,   7.778553  ],\n",
       "       [  7.3694544 ,   7.4603963 ,   4.0450187 ],\n",
       "       [  6.761831  ,   6.2402434 ,   3.6689937 ],\n",
       "       [  7.1449757 ,   5.06627   ,   4.377878  ],\n",
       "       [  8.154776  ,   5.1049476 ,   5.3690214 ],\n",
       "       [  8.311182  ,   7.491954  ,   5.0833883 ],\n",
       "       [  8.673593  ,   6.345044  ,   5.744974  ],\n",
       "       [  9.279375  ,  -1.4066955 ,  -1.1303563 ],\n",
       "       [  9.695962  ,  -0.84360904,   0.09346329],\n",
       "       [ 10.741167  ,  -1.4047068 ,   0.85026306],\n",
       "       [ 11.337838  ,  -2.5152693 ,   0.34818384],\n",
       "       [  9.947194  ,  -2.503446  ,  -1.6427891 ],\n",
       "       [ 10.963412  ,  -3.0920165 ,  -0.8827657 ],\n",
       "       [ -2.7861156 ,   3.6136692 ,  10.156345  ],\n",
       "       [ -2.2059588 ,   4.6645126 ,  10.859446  ],\n",
       "       [ -2.8828313 ,   5.2815237 ,  11.878384  ],\n",
       "       [ -4.051571  ,   4.703183  ,  12.398331  ],\n",
       "       [ -4.029222  ,   3.0349517 ,  10.627197  ],\n",
       "       [ -4.625328  ,   3.5627646 ,  11.798974  ],\n",
       "       [ -6.6532946 ,   0.6536499 ,  -1.7581149 ],\n",
       "       [ -5.434176  ,   0.80878127,  -1.010796  ],\n",
       "       [ -5.269722  ,   1.969161  ,  -0.30620632],\n",
       "       [ -6.162678  ,   3.0661705 ,  -0.41737133],\n",
       "       [ -7.554871  ,   1.638897  ,  -1.8851368 ],\n",
       "       [ -7.2567034 ,   2.8887343 ,  -1.2123115 ],\n",
       "       [ -7.1318502 ,   9.973084  , -22.845293  ],\n",
       "       [ -7.9938297 ,  11.093233  , -22.600224  ],\n",
       "       [ -7.656463  ,  12.352939  , -22.940662  ],\n",
       "       [ -6.3405275 ,  12.600382  , -23.369509  ],\n",
       "       [ -5.8314853 ,  10.2353525 , -23.233128  ],\n",
       "       [ -5.4221644 ,  11.56776   , -23.510313  ],\n",
       "       [-14.47144   ,   3.9830515 , -16.958258  ],\n",
       "       [-15.724198  ,   3.4074953 , -17.121016  ],\n",
       "       [-15.925173  ,   2.07661   , -16.701107  ],\n",
       "       [-14.910878  ,   1.3488185 , -15.954925  ],\n",
       "       [-13.429387  ,   3.2457547 , -16.299551  ],\n",
       "       [-13.708363  ,   1.9560267 , -15.797147  ],\n",
       "       [  0.68489426,  -7.1988363 ,  -7.9329753 ],\n",
       "       [  1.6239585 ,  -6.234926  ,  -8.350962  ],\n",
       "       [  1.8822813 ,  -5.032412  ,  -7.6419168 ],\n",
       "       [  1.1307101 ,  -4.8801293 ,  -6.4649773 ],\n",
       "       [ -0.07314643,  -6.9601626 ,  -6.7442245 ],\n",
       "       [  0.18348259,  -5.8281007 ,  -5.9979796 ],\n",
       "       [  5.240214  ,  -9.8966675 ,   7.958326  ],\n",
       "       [  5.138345  ,  -8.475347  ,   8.108525  ],\n",
       "       [  6.0967474 ,  -7.774202  ,   8.737614  ],\n",
       "       [  7.218897  ,  -8.407282  ,   9.168674  ],\n",
       "       [  6.4802237 , -10.59063   ,   8.378847  ],\n",
       "       [  7.4003906 ,  -9.800496  ,   9.041461  ],\n",
       "       [  1.7539014 ,  -6.0086555 ,   6.441855  ],\n",
       "       [  2.7657003 ,  -5.3664594 ,   7.2473598 ],\n",
       "       [  3.898067  ,  -4.751381  ,   6.659806  ],\n",
       "       [  3.9854598 ,  -4.80297   ,   5.240488  ],\n",
       "       [  1.7967764 ,  -5.9690633 ,   5.0908628 ],\n",
       "       [  2.9536502 ,  -5.3387427 ,   4.513858  ],\n",
       "       [ -4.7725525 ,  -4.6910343 ,   8.015858  ],\n",
       "       [ -5.9240456 ,  -4.6293626 ,   7.175289  ],\n",
       "       [ -5.797645  ,  -5.129216  ,   5.8476315 ],\n",
       "       [ -4.6323576 ,  -5.7940083 ,   5.429229  ],\n",
       "       [ -3.6474338 ,  -5.4907613 ,   7.6195135 ],\n",
       "       [ -3.5659113 ,  -6.0071363 ,   6.319044  ]], dtype=float32)"
      ]
     },
     "execution_count": 52,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "Ca = u.select_atoms('type CA')\n",
    "\n",
    "Ca.positions"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This returns a `numpy` array, which can be manipulated as you have previously learnt."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Challenge: \n",
    "\n",
    "- #### calculate the center of geometry of the C$_\\alpha$ atoms\n",
    "- #### select all atoms that are below the plane x=4.0"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4.1 Trajectory manipulation\n",
    "\n",
    "Only a single frame of trajectory information is accessible from an `AtomGroup` at any one time,\n",
    "a `Universe` is created with the first frame of trajectory information loaded.\n",
    "\n",
    "To access data from a different frame, we must manipulate the `Universe.trajectory` object."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 40,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<DCDReader /Users/richardgowers/MDAnalysis_data/adk_equilibrium/1ake_007-nowater-core-dt240ps.dcd with 4187 frames of 3341 atoms>"
      ]
     },
     "execution_count": 40,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.trajectory"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "By slicing the `.trajectory` attribute of a `Universe`, we change the currently loaded frame"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 41,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "< Timestep 1 with unit cell dimensions [85.69929 85.69929 85.69929 90.      90.      90.     ] >\n",
      "< Timestep 24 with unit cell dimensions [85.72862 85.72862 85.72862 90.      90.      90.     ] >\n"
     ]
    }
   ],
   "source": [
    "print(u.trajectory[1])\n",
    "print(u.trajectory[24])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "After we have changed the currently loaded frame, the data we access from an `AtomGroup` will now correspond to the data from that frame:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 42,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[1.7000737 5.001091  7.855839 ]\n",
      "[2.6505709 4.418251  8.170883 ]\n",
      "[2.2835758 4.4647355 8.492147 ]\n"
     ]
    }
   ],
   "source": [
    "atom = u.atoms[42]\n",
    "\n",
    "u.trajectory[0]\n",
    "print(atom.position)\n",
    "\n",
    "u.trajectory[1]\n",
    "print(atom.position)\n",
    "\n",
    "u.trajectory[2]\n",
    "print(atom.position)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A more common pattern for moving through the trajectory data is to slice the `.trajectory` object inside a `for` loop:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 43,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[1.7000737 5.001091  7.855839 ]\n",
      "[2.6505709 4.418251  8.170883 ]\n",
      "[2.2835758 4.4647355 8.492147 ]\n",
      "[2.2192698 4.3338423 8.107021 ]\n",
      "[2.5894134 4.7559743 8.439111 ]\n"
     ]
    }
   ],
   "source": [
    "for ts in u.trajectory[:5]:\n",
    "    print(atom.position)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The trajectory can be sliced in any way that a `numpy` array or `AtomGroup` can be sliced, for example:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 44,
   "metadata": {},
   "outputs": [],
   "source": [
    "for ts in u.trajectory[::10]:  # every 10th frame\n",
    "    pass\n",
    "\n",
    "for ts in u.trajectory[[0, 5, 17]]: # pick out individual frames\n",
    "    pass"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Challenges\n",
    "\n",
    "- #### How could you iterate through the trajectory backwards?\n",
    "\n",
    "\n",
    "- #### How could you slice the trajectory to correspond to when the system time is after 600 ps?"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4.2 The `Timestep` object\n",
    "\n",
    "When the trajectory is sliced, it returns a `Timestep` object.\n",
    "This object represents the data in the currently loaded trajectory frame.\n",
    "\n",
    "It has some useful attributes such as:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 45,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "10\n",
      "2400.0000489999024\n",
      "240.00000489999024\n",
      "False\n"
     ]
    }
   ],
   "source": [
    "ts = u.trajectory[10]\n",
    "\n",
    "print(ts.frame)\n",
    "\n",
    "print(ts.time)\n",
    "\n",
    "print(u.trajectory.dt)\n",
    "\n",
    "print(ts.has_velocities)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 5 Bonds, angles, and dihedrals\n",
    "\n",
    "We can also get at connectivity information between atoms, such as bonds, angles, and dihedrals "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 41,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<TopologyGroup containing 3365 bonds>"
      ]
     },
     "execution_count": 41,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms.bonds"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 42,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<TopologyGroup containing 6123 angles>"
      ]
     },
     "execution_count": 42,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms.angles"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 43,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<TopologyGroup containing 8921 dihedrals>"
      ]
     },
     "execution_count": 43,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms.dihedrals"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 38,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<Bond between: Atom 0, Atom 4>"
      ]
     },
     "execution_count": 38,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.bonds[3]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Want the actual value?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 39,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "1.4905602"
      ]
     },
     "execution_count": 39,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.bonds[3].value()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 46,
   "metadata": {},
   "outputs": [],
   "source": [
    "import matplotlib.pyplot as plt\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 47,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(array([2.300e+01, 2.950e+02, 5.700e+01, 2.900e+02, 1.019e+03, 0.000e+00,\n",
       "        1.000e+00, 3.800e+01, 1.260e+02, 1.020e+02, 4.500e+01, 6.200e+01,\n",
       "        1.330e+02, 9.400e+01, 8.200e+01, 1.130e+02, 1.430e+02, 2.190e+02,\n",
       "        2.460e+02, 1.830e+02, 7.000e+01, 1.000e+01, 1.000e+00, 0.000e+00,\n",
       "        0.000e+00, 0.000e+00, 2.000e+00, 1.000e+00, 4.000e+00, 6.000e+00]),\n",
       " array([0.95999761, 0.99100672, 1.02201584, 1.05302495, 1.08403407,\n",
       "        1.11504319, 1.1460523 , 1.17706142, 1.20807053, 1.23907965,\n",
       "        1.27008876, 1.30109788, 1.332107  , 1.36311611, 1.39412523,\n",
       "        1.42513434, 1.45614346, 1.48715257, 1.51816169, 1.5491708 ,\n",
       "        1.58017992, 1.61118904, 1.64219815, 1.67320727, 1.70421638,\n",
       "        1.7352255 , 1.76623461, 1.79724373, 1.82825284, 1.85926196,\n",
       "        1.89027108]),\n",
       " <a list of 30 Patch objects>)"
      ]
     },
     "execution_count": 47,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYAAAAD8CAYAAAB+UHOxAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDIuMi4yLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvhp/UCwAADwhJREFUeJzt3X+snmddx/H3x5UhP5SO9TBnWz1TKj8kLGAdU/yBqwE6CJ0JIyBhZWnSP5yIzsRVY1wi/rEl6hhRRppttksIjIzFVZmQZYBocJMO5garSDPmetykB1eGshAt+/rHcxUO3ek5Z89zzvO053q/kpNz39d9Pc/9PVdOn8+5r/tHU1VIkvrzA5MuQJI0GQaAJHXKAJCkThkAktQpA0CSOmUASFKnDABJ6pQBIEmdMgAkqVNrJl3AQtatW1fT09OTLkOSTin33HPP16tqarF+J3UATE9Ps3///kmXIUmnlCT/vpR+TgFJUqcMAEnqlAEgSZ0yACSpUwaAJHXKAJCkThkAktQpA0CSOmUASFKnFr0TOMmNwBuBw1X1stb2fOBmYBp4CHhLVR1JEuBa4ELgCeCdVfX59prtwB+2t/2Tqtq7vD/KqWd618eW1O+hq96wwpVI6tFSjgD2AK8/rm0XcGdVbQLubOsAW4FN7WsncB18NzCuBF4FnAdcmeSMUYuXJA1v0QCoqs8Ajx3XvA049hf8XuCiOe031cBdwNokZwOvA+6oqseq6ghwB08NFUnSGA17DuCsqnoUoH1/QWtfDxya02+mtZ2oXZI0Ict9EjjztNUC7U99g2Rnkv1J9s/Ozi5rcZKk7xk2AL7WpnZo3w+39hlg45x+G4BHFmh/iqraXVWbq2rz1NSij7OWJA1p2ADYB2xvy9uB2+a0X5KB84HH2xTRJ4DXJjmjnfx9bWuTJE3IUi4D/RDwGmBdkhkGV/NcBXwkyQ7gYeDi1v12BpeAHmRwGeilAFX1WJL3AJ9r/f64qo4/sSxJGqNFA6Cq3naCTVvm6VvAZSd4nxuBG59WdZKkFeOdwJLUKQNAkjplAEhSpwwASeqUASBJnTIAJKlTBoAkdcoAkKROGQCS1CkDQJI6ZQBIUqcMAEnqlAEgSZ0yACSpUwaAJHXKAJCkThkAktQpA0CSOmUASFKnDABJ6pQBIEmdMgAkqVMGgCR1ygCQpE4ZAJLUKQNAkjplAEhSpwwASeqUASBJnTIAJKlTBoAkdcoAkKROjRQASX4nyZeSfDHJh5L8YJJzktyd5CtJbk5yeuv7zLZ+sG2fXo4fQJI0nKEDIMl64LeAzVX1MuA04K3A1cA1VbUJOALsaC/ZARypqhcC17R+kqQJGXUKaA3wrCRrgGcDjwIXALe07XuBi9rytrZO274lSUbcvyRpSEMHQFX9B/CnwMMMPvgfB+4BvlFVR1u3GWB9W14PHGqvPdr6n3n8+ybZmWR/kv2zs7PDlidJWsQoU0BnMPir/hzgR4HnAFvn6VrHXrLAtu81VO2uqs1VtXlqamrY8iRJixhlCuhXga9W1WxV/R9wK/DzwNo2JQSwAXikLc8AGwHa9ucBj42wf0nSCEYJgIeB85M8u83lbwEeAD4FvLn12Q7c1pb3tXXa9k9W1VOOACRJ4zHKOYC7GZzM/Txwf3uv3cAVwOVJDjKY47+hveQG4MzWfjmwa4S6JUkjWrN4lxOrqiuBK49rfhA4b56+3wYuHmV/kqTl453AktQpA0CSOmUASFKnDABJ6pQBIEmdMgAkqVMGgCR1ygCQpE4ZAJLUKQNAkjplAEhSpwwASeqUASBJnTIAJKlTBoAkdcoAkKROGQCS1CkDQJI6ZQBIUqcMAEnqlAEgSZ0yACSpUwaAJHXKAJCkThkAktQpA0CSOmUASFKnDABJ6pQBIEmdMgAkqVMGgCR1aqQASLI2yS1J/jXJgSQ/l+T5Se5I8pX2/YzWN0nel+RgkvuSvHJ5fgRJ0jBGPQK4Fvh4Vb0YOBc4AOwC7qyqTcCdbR1gK7Cpfe0Erhtx35KkEQwdAEl+GPgl4AaAqvrfqvoGsA3Y27rtBS5qy9uAm2rgLmBtkrOHrlySNJJRjgB+ApgF/irJF5Jcn+Q5wFlV9ShA+/6C1n89cGjO62damyRpAkYJgDXAK4HrquoVwLf43nTPfDJPWz2lU7Izyf4k+2dnZ0coT5K0kFECYAaYqaq72/otDALha8emdtr3w3P6b5zz+g3AI8e/aVXtrqrNVbV5ampqhPIkSQsZOgCq6j+BQ0le1Jq2AA8A+4DtrW07cFtb3gdc0q4GOh94/NhUkSRp/NaM+Pp3AR9McjrwIHApg1D5SJIdwMPAxa3v7cCFwEHgidZXkjQhIwVAVd0LbJ5n05Z5+hZw2Sj7kyQtH+8ElqROGQCS1CkDQJI6ZQBIUqcMAEnqlAEgSZ0yACSpUwaAJHXKAJCkThkAktQpA0CSOmUASFKnDABJ6pQBIEmdMgAkqVMGgCR1ygCQpE4ZAJLUKQNAkjplAEhSpwwASeqUASBJnTIAJKlTBoAkdcoAkKROGQCS1CkDQJI6ZQBIUqcMAEnqlAEgSZ0yACSpUwaAJHVq5ABIclqSLyT527Z+TpK7k3wlyc1JTm/tz2zrB9v26VH3LUka3nIcAbwbODBn/WrgmqraBBwBdrT2HcCRqnohcE3rJ0makJECIMkG4A3A9W09wAXALa3LXuCitrytrdO2b2n9JUkTMOoRwHuB3wOebOtnAt+oqqNtfQZY35bXA4cA2vbHW39J0gQMHQBJ3ggcrqp75jbP07WWsG3u++5Msj/J/tnZ2WHLkyQtYpQjgFcDb0ryEPBhBlM/7wXWJlnT+mwAHmnLM8BGgLb9ecBjx79pVe2uqs1VtXlqamqE8iRJCxk6AKrq96tqQ1VNA28FPllVbwc+Bby5ddsO3NaW97V12vZPVtVTjgAkSeOxEvcBXAFcnuQggzn+G1r7DcCZrf1yYNcK7FuStERrFu+yuKr6NPDptvwgcN48fb4NXLwc+5Mkjc47gSWpUwaAJHXKAJCkThkAktQpA0CSOmUASFKnDABJ6pQBIEmdMgAkqVMGgCR1ygCQpE4ZAJLUKQNAkjplAEhSpwwASeqUASBJnVqW/xDmVDe962NL6vfQVW9Y4UokaXwMgBWw1ECRpElyCkiSOmUASFKnDABJ6pQBIEmdMgAkqVMGgCR1ygCQpE4ZAJLUKW8Ek04x3rmu5eIRgCR1ygCQpE4ZAJLUKQNAkjo19EngJBuBm4AfAZ4EdlfVtUmeD9wMTAMPAW+pqiNJAlwLXAg8Abyzqj4/WvnS6uFTZDVuoxwBHAV+t6peApwPXJbkpcAu4M6q2gTc2dYBtgKb2tdO4LoR9i1JGtHQAVBVjx77C76q/hs4AKwHtgF7W7e9wEVteRtwUw3cBaxNcvbQlUuSRrIs5wCSTAOvAO4GzqqqR2EQEsALWrf1wKE5L5tpbZKkCRg5AJI8F/go8NtV9c2Fus7TVvO8384k+5Psn52dHbU8SdIJjBQASZ7B4MP/g1V1a2v+2rGpnfb9cGufATbOefkG4JHj37OqdlfV5qraPDU1NUp5kqQFDB0A7aqeG4ADVfXnczbtA7a35e3AbXPaL8nA+cDjx6aKJEnjN8qzgF4NvAO4P8m9re0PgKuAjyTZATwMXNy23c7gEtCDDC4DvXSEfUuSRjR0AFTVPzL/vD7Alnn6F3DZsPuT9PT40DgtxjuBJalTBoAkdcoAkKROGQCS1CkDQJI6ZQBIUqcMAEnqlP8pvLTCfM6/TlYeAUhSpzwC0CnNu12l4RkAWpAfsNLq5RSQJHXKAJCkTjkFJA3Jq3t0qvMIQJI65RGAloUni6VTj0cAktQpjwDUhaczX+9RinrhEYAkdcojAOk4Xt2jXngEIEmdMgAkqVNOAUmd8wR5vwwAnZSch5dWngHwNPihJGk18RyAJHXKAJCkTjkFpLFyGk06eRgAnfKDWJJTQJLUKQNAkjq1qqeAnOaQpBMb+xFAktcn+XKSg0l2jXv/kqSBsQZAktOAvwS2Ai8F3pbkpeOsQZI0MO4poPOAg1X1IECSDwPbgAfGXIekIfhff64u4w6A9cChOeszwKvGXIOkzq1EkC33OcdxhOi4AyDztNX3dUh2Ajvb6v8k+fKKV/X91gFfH/M+F5Srx77Lk24MJsRxGHIMJvA7uyLazzGR34MRx/DHl9Jp3AEwA2ycs74BeGRuh6raDeweZ1FzJdlfVZsntf+TgWMw4Dg4BrC6x2DcVwF9DtiU5JwkpwNvBfaNuQZJEmM+Aqiqo0l+E/gEcBpwY1V9aZw1SJIGxn4jWFXdDtw+7v0+DRObfjqJOAYDjoNjAKt4DFJVi/eSJK06PgtIkjrVZQAkuTHJ4SRfPMH2JHlfe1zFfUleOe4aV9oSxuDt7We/L8lnk5w77hrHYbFxmNPvZ5N8J8mbx1XbuCxlDJK8Jsm9Sb6U5O/HWd+4LOHfxPOS/E2Sf2njcOm4a1xuXQYAsAd4/QLbtwKb2tdO4Lox1DRue1h4DL4K/HJVvRx4D6t3HnQPC4/DsUeYXM3g4oXVaA8LjEGStcD7gTdV1U8DF4+prnHbw8K/C5cBD1TVucBrgD9rVzOesroMgKr6DPDYAl22ATfVwF3A2iRnj6e68VhsDKrqs1V1pK3exeCejVVnCb8LAO8CPgocXvmKxm8JY/DrwK1V9XDr3+s4FPBDSQI8t/U9Oo7aVkqXAbAE8z2yYv2EajkZ7AD+btJFTEKS9cCvAR+YdC0T9FPAGUk+neSeJJdMuqAJ+QvgJQxuXr0feHdVPTnZkkazqv8/gBEs+siKXiT5FQYB8AuTrmVC3gtcUVXfGfzh16U1wM8AW4BnAf+U5K6q+rfJljV2rwPuBS4AfhK4I8k/VNU3J1vW8AyA+S36yIoeJHk5cD2wtar+a9L1TMhm4MPtw38dcGGSo1X115Mta6xmgK9X1beAbyX5DHAu0FsAXApcVYNr5w8m+SrwYuCfJ1vW8JwCmt8+4JJ2NdD5wONV9eikixqnJD8G3Aq8o8O/9L6rqs6pqumqmgZuAX6jsw9/gNuAX0yyJsmzGTzB98CEa5qEhxkcBZHkLOBFwIMTrWhEXR4BJPkQg7P465LMAFcCzwCoqg8wuFP5QuAg8ASD5F9VljAGfwScCby//fV7dDU+EGsJ47DqLTYGVXUgyceB+4AngeurasHLZk9FS/hdeA+wJ8n9DKaJr6iqU/ppsd4JLEmdcgpIkjplAEhSpwwASeqUASBJnTIAJKlTBoAkdcoAkKROGQCS1Kn/By9E1A7jwE9DAAAAAElFTkSuQmCC\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.hist(u.bonds.values(), bins=30)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "These work the same way as `AtomGroup`s. They're sliceable, and indexing them works too to give individual bonds, angles, dihedrals."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Special Functions\n",
    "\n",
    "The `split` function can be used to return a list of atomgroups that are separated by the specified level. For example `u.atoms.split('residue')` will return a list of all residues as atomgroups."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 55,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[<AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 20 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 21 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 11 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 11 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 11 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 11 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 20 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 20 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 21 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 20 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 11 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 21 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 20 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 21 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 17 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 21 atoms>,\n",
       " <AtomGroup with 21 atoms>,\n",
       " <AtomGroup with 11 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 21 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 7 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 14 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 16 atoms>,\n",
       " <AtomGroup with 24 atoms>,\n",
       " <AtomGroup with 10 atoms>,\n",
       " <AtomGroup with 12 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 15 atoms>,\n",
       " <AtomGroup with 22 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 19 atoms>,\n",
       " <AtomGroup with 8 atoms>]"
      ]
     },
     "execution_count": 55,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms.split('residue')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Another method is `groupby`. You use it to select for a topolgy attribute (*resnames*, *names*, *masses*, ...). `groupby` will return a dictionary where the keys are all unique values of a given topology attribute and the values are a group of atoms that have this value."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 57,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'ARG': <AtomGroup with 312 atoms>,\n",
       " 'GLU': <AtomGroup with 270 atoms>,\n",
       " 'VAL': <AtomGroup with 304 atoms>,\n",
       " 'ALA': <AtomGroup with 190 atoms>,\n",
       " 'SER': <AtomGroup with 55 atoms>,\n",
       " 'PHE': <AtomGroup with 100 atoms>,\n",
       " 'LYS': <AtomGroup with 396 atoms>,\n",
       " 'GLN': <AtomGroup with 136 atoms>,\n",
       " 'TYR': <AtomGroup with 147 atoms>,\n",
       " 'PRO': <AtomGroup with 140 atoms>,\n",
       " 'THR': <AtomGroup with 154 atoms>,\n",
       " 'CYS': <AtomGroup with 11 atoms>,\n",
       " 'GLY': <AtomGroup with 141 atoms>,\n",
       " 'ASP': <AtomGroup with 204 atoms>,\n",
       " 'ASN': <AtomGroup with 56 atoms>,\n",
       " 'HSE': <AtomGroup with 51 atoms>,\n",
       " 'MET': <AtomGroup with 104 atoms>,\n",
       " 'LEU': <AtomGroup with 304 atoms>,\n",
       " 'ILE': <AtomGroup with 266 atoms>}"
      ]
     },
     "execution_count": 57,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms.groupby('resnames')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Multiple topology attributes can also be given."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 61,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{('ARG', 'O'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HN'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HG1'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'CA'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HB1'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HG2'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HD2'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'C'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HH12'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'CG'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'N'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HA'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'NH2'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'NH1'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HD1'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HE'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'CZ'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HH21'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HH11'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HH22'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'NE'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'CB'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'HB2'): <AtomGroup with 13 atoms>,\n",
       " ('ARG', 'CD'): <AtomGroup with 13 atoms>,\n",
       " ('GLU', 'HB1'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'N'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'O'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'HA'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'HG2'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'C'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'HN'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'CB'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'OE2'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'OE1'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'HG1'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'HB2'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'CD'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'CG'): <AtomGroup with 18 atoms>,\n",
       " ('GLU', 'CA'): <AtomGroup with 18 atoms>,\n",
       " ('VAL', 'N'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'CG1'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HG23'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HA'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'O'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'CG2'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'C'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HG11'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HN'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'CB'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HG13'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HG12'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HB'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HG21'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'HG22'): <AtomGroup with 19 atoms>,\n",
       " ('VAL', 'CA'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'HB1'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'N'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'O'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'HA'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'C'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'HN'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'CB'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'HB2'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'HB3'): <AtomGroup with 19 atoms>,\n",
       " ('ALA', 'CA'): <AtomGroup with 19 atoms>,\n",
       " ('SER', 'OG'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'HB1'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'N'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'O'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'HA'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'C'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'HN'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'CB'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'HB2'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'HG1'): <AtomGroup with 5 atoms>,\n",
       " ('SER', 'CA'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HD1'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HB1'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'N'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HE1'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CE1'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HA'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CZ'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HZ'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HD2'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HE2'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'C'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'O'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CE2'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HN'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CB'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'HB2'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CD1'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CD2'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CG'): <AtomGroup with 5 atoms>,\n",
       " ('PHE', 'CA'): <AtomGroup with 5 atoms>,\n",
       " ('LYS', 'O'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HN'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'CD'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HG1'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'CA'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HB1'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HZ2'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HG2'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HD2'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'C'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'NZ'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'CG'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HE2'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'N'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'CE'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HA'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HZ1'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HD1'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HZ3'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'CB'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HB2'): <AtomGroup with 18 atoms>,\n",
       " ('LYS', 'HE1'): <AtomGroup with 18 atoms>,\n",
       " ('GLN', 'HB1'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'N'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'HE21'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'O'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'HA'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'HG2'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'C'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'HN'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'CB'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'OE1'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'HG1'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'HB2'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'CD'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'NE2'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'HE22'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'CG'): <AtomGroup with 8 atoms>,\n",
       " ('GLN', 'CA'): <AtomGroup with 8 atoms>,\n",
       " ('TYR', 'CE1'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'O'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HN'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'OH'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'CA'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HB1'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HD2'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'C'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'CD2'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'CG'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HE2'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HH'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'N'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HA'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'CD1'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HD1'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'CZ'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'CE2'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'CB'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HB2'): <AtomGroup with 7 atoms>,\n",
       " ('TYR', 'HE1'): <AtomGroup with 7 atoms>,\n",
       " ('PRO', 'HD1'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'HB1'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'N'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'O'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'HA'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'HG2'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'HD2'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'C'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'CB'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'HG1'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'CD'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'HB2'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'CG'): <AtomGroup with 10 atoms>,\n",
       " ('PRO', 'CA'): <AtomGroup with 10 atoms>,\n",
       " ('THR', 'N'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'HG23'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'HA'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'O'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'CG2'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'C'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'HN'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'HG22'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'CB'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'OG1'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'HB'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'HG21'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'HG1'): <AtomGroup with 11 atoms>,\n",
       " ('THR', 'CA'): <AtomGroup with 11 atoms>,\n",
       " ('CYS', 'HB1'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'N'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'O'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'HA'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'C'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'HN'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'CB'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'HB2'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'SG'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'HG1'): <AtomGroup with 1 atom>,\n",
       " ('CYS', 'CA'): <AtomGroup with 1 atom>,\n",
       " ('GLY', 'N'): <AtomGroup with 20 atoms>,\n",
       " ('GLY', 'OT1'): <AtomGroup with 1 atom>,\n",
       " ('GLY', 'O'): <AtomGroup with 19 atoms>,\n",
       " ('GLY', 'OT2'): <AtomGroup with 1 atom>,\n",
       " ('GLY', 'C'): <AtomGroup with 20 atoms>,\n",
       " ('GLY', 'HN'): <AtomGroup with 20 atoms>,\n",
       " ('GLY', 'HA2'): <AtomGroup with 20 atoms>,\n",
       " ('GLY', 'CA'): <AtomGroup with 20 atoms>,\n",
       " ('GLY', 'HA1'): <AtomGroup with 20 atoms>,\n",
       " ('ASP', 'HB1'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'N'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'O'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'HA'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'C'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'OD1'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'HN'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'CB'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'HB2'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'OD2'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'CG'): <AtomGroup with 17 atoms>,\n",
       " ('ASP', 'CA'): <AtomGroup with 17 atoms>,\n",
       " ('ASN', 'HB1'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'N'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'O'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'HA'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'HD22'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'HD21'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'C'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'ND2'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'OD1'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'HN'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'CB'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'HB2'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'CG'): <AtomGroup with 4 atoms>,\n",
       " ('ASN', 'CA'): <AtomGroup with 4 atoms>,\n",
       " ('HSE', 'HB1'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'N'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'ND1'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'CE1'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'HA'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'HE1'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'HE2'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'HD2'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'O'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'C'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'HN'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'CB'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'NE2'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'HB2'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'CD2'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'CG'): <AtomGroup with 3 atoms>,\n",
       " ('HSE', 'CA'): <AtomGroup with 3 atoms>,\n",
       " ('MET', 'HB1'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'N'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'SD'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'CE'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HA'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HG2'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HE1'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HE2'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HE3'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'C'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HT2'): <AtomGroup with 1 atom>,\n",
       " ('MET', 'O'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HT1'): <AtomGroup with 1 atom>,\n",
       " ('MET', 'HN'): <AtomGroup with 5 atoms>,\n",
       " ('MET', 'CB'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HG1'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'HT3'): <AtomGroup with 1 atom>,\n",
       " ('MET', 'HB2'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'CG'): <AtomGroup with 6 atoms>,\n",
       " ('MET', 'CA'): <AtomGroup with 6 atoms>,\n",
       " ('LEU', 'HB1'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'N'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HD23'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HA'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HD11'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HD22'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HD21'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'O'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'C'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HN'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'CB'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HG'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HD12'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HD13'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'HB2'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'CD1'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'CD2'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'CG'): <AtomGroup with 16 atoms>,\n",
       " ('LEU', 'CA'): <AtomGroup with 16 atoms>,\n",
       " ('ILE', 'HD1'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'N'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'CG1'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HG23'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HA'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HD3'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HD2'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'O'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'CG2'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'C'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HG11'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HN'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'CB'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HG12'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HB'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'CD'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HG21'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'HG22'): <AtomGroup with 14 atoms>,\n",
       " ('ILE', 'CA'): <AtomGroup with 14 atoms>}"
      ]
     },
     "execution_count": 61,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "u.atoms.groupby(['resnames', 'names'])"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
© 2021 GitHub, Inc.
Terms
Privacy
Security
Status
Docs
Contact GitHub
Pricing
API
Training
Blog
About
